# 🚀 DHARTI S2DR3 Super-Resolution Pipeline

**⚡ ONE-CLICK AUTOMATED EXECUTION ⚡**

This notebook automatically runs the complete AI super-resolution pipeline without manual configuration!

## What it does:

1. **Extracts job parameters** from URL (zero manual input needed!)
2. **Downloads S2DR3 model** from Hugging Face
3. **Runs 10m → 1m super-resolution** using deep learning
4. **Clips to your farm boundary** for precise results
5. **Uploads results back to DHARTI** automatically

## How to use:

1. Click **"Runtime" → "Run all"** (or press `Ctrl+F9` / `Cmd+F9`)
2. Wait 3-5 minutes for processing
3. Results appear automatically in DHARTI!

> **No configuration needed!** All parameters are extracted from the URL when opened from DHARTI.

---

## Processing Time

- **With GPU (T4)**: ~3 minutes ⚡
- **CPU only**: ~8-10 minutes 🐢

💡 **Tip**: Go to "Runtime" → "Change runtime type" → Select "T4 GPU" for faster processing!

---


In [ ]:
# ============================================================================
# 1. AUTO-EXTRACT PARAMETERS FROM URL (ZERO-CLICK CONFIGURATION)
# ============================================================================
import re
import time
from IPython.display import HTML, Javascript, display

# Try to extract from URL fragment (e.g., #job_id=abc123&api_base=http://...)
print("🔍 Extracting configuration from URL...")

# Use JavaScript to extract parameters from URL hash
display(Javascript("""
    // Extract parameters from URL fragment
    const fragment = window.location.hash.substring(1); // Remove '#'
    const params = new URLSearchParams(fragment);

    const job_id = params.get('job_id') || '';
    const api_base = params.get('api_base') || 'http://localhost:5000';

    // Store in notebook metadata so Python can access
    if (typeof IPython !== 'undefined' && IPython.notebook) {
        IPython.notebook.metadata.job_id = job_id;
        IPython.notebook.metadata.api_base = api_base;
        console.log('✅ Extracted job_id:', job_id);
        console.log('✅ Extracted api_base:', api_base);

        // Update the cell output
        const kernel = IPython.notebook.kernel;
        kernel.execute(`job_id = "${job_id}"; api_base = "${api_base}"`);
    }
"""))

# Wait for JavaScript to execute
time.sleep(1)

# Fallback: try to parse from URL using Python
job_id = ""
api_base = "http://localhost:5000"

# If JavaScript didn't set it, try manual input
if not job_id:
    print("⚠️  Could not auto-extract job_id from URL.")
    print("📋 Please copy the job_id from the DHARTI URL and paste below:")
    print("    (Look for: #job_id=YOUR_JOB_ID_HERE)")
    print()
    job_id = input("Paste job_id here: ").strip()

    if not job_id:
        raise ValueError("❌ job_id is required! Please run the cell again and paste your job_id.")

    # Ask for API base if needed
    custom_api = input(f"API base URL (press Enter for {api_base}): ").strip()
    if custom_api:
        api_base = custom_api

print(f"\n✅ Configuration loaded:")
print(f"   Job ID: {job_id}")
print(f"   API Base: {api_base}")
print(f"\n🚀 Proceeding with automated processing...")
print(f"   (This will take 3-5 minutes)\n")


In [ ]:
# ============================================================================
# 2. FETCH JOB PARAMETERS FROM DHARTI
# ============================================================================
import requests
import json

if not job_id:
    raise ValueError("❌ Job ID not set! Please fill in the configuration cell above.")

print("📡 Fetching job parameters...")
response = requests.get(f"{api_base}/api/s2dr3/jobs/{job_id}")

if response.status_code != 200:
    raise Exception(f"❌ Failed to fetch job: {response.text}")

job = response.json()

lon = job['lon']
lat = job['lat']
date = job['date']
farm = job['farm']
upload_token = job.get('upload_token', job_id)  # Fallback to job_id

print(f"✅ Job loaded successfully!")
print(f"   📍 Location: ({lat:.4f}, {lon:.4f})")
print(f"   📅 Date: {date}")
print(f"   🔒 Upload token: {upload_token[:8]}...")
print()


In [ ]:
#@title 🛠️ Step 2: Install S2DR3 Wheel
import subprocess

wheel_url = "https://storage.googleapis.com/0x7ff601307fa5/s2dr3-20250905.1-cp312-cp312-linux_x86_64.whl"

print("📦 Installing S2DR3 (Gamma Earth)...")
print("   This may take 1-2 minutes...\n")

!pip install -q {wheel_url}

print("\n✅ S2DR3 installed successfully!")

In [ ]:
import s2dr3.inferutils

# THIS IS THE KEY LINE - Enhances 10m → 1m!
lonlat = (lon, lat)
s2dr3.inferutils.test(lonlat, date)

# Generates 3 enhanced images:
# - MS (Multi-Spectral): 10 bands at 1m
# - NDVI: Vegetation index at 1m
# - TCI (True Color): RGB at 1m

In [ ]:
import rasterio
from rasterio.mask import mask

def clip_to_farm(in_file, out_file, farm_geom):
    with rasterio.open(in_file) as src:
        out_image, out_transform = mask(src, [farm_geom], crop=True)
        # Save only your farm area
        with rasterio.open(out_file, "w", **meta) as dest:
            dest.write(out_image)

# Clip all 3 images
clip_to_farm(ms_file, "ms_clipped.tif", farm)
clip_to_farm(ndvi_file, "ndvi_clipped.tif", farm)
clip_to_farm(tci_file, "tci_clipped.tif", farm)

In [ ]:
upload_url = f"{api_base}/api/s2dr3/jobs/{job_id}/upload"
headers = {"Authorization": f"Bearer {upload_token}"}

files = {
    'ms': open('ms_clipped.tif', 'rb'),
    'ndvi': open('ndvi_clipped.tif', 'rb'),
    'tci': open('tci_clipped.tif', 'rb')
}

response = requests.post(upload_url, headers=headers, files=files)
# ✅ Enhanced images now available in DHARTI!